# TP Metric learning

Save the notebook as either PDF or HTML and make sure all the results are saved correctly (I won't run them and the original format does not save the results automatically), **and put your name in the filename**.

<div class="alert alert-success"> 
<b>Questions are in green boxes.</b>
The maximum time you should spend on each question is given as indication only. If you take more time than that, then you should come see me.
</div>
<div class="alert alert-info" role="alert"><b>Analyzes are in blue boxes.</b> You should comment on your results in theses boxes (Is it good? Is it expected? Why do we get such result? Why is it different from the previous one? etc)
</div>

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import gzip
import pickle
import matplotlib.pyplot as plt
from functools import partial

For this lab, we will use the [MNIST dataset (~15Mo)](https://educnet.enpc.fr/mod/resource/view.php?id=44975). It consists of 28x28 images (loaded as a 784 vector) and the associated label for training, validation and test sets. 

The following code loads all samples from the MNIST archive into several arrays, then displays the first sample from the training set as an image. We will limit ourselves to the first 1000 samples of train and 1000 samples of validation.

In [ ]:
# Load the dataset
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, val_set, test_set = pickle.load(f, encoding='latin1') # python 3
# train_set, val_set, test_set = cPickle.load(f) # python 2
f.close()
train_data=train_set[0][0:1000, :]
train_labels=train_set[1][0:1000]
val_data=val_set[0][0:1000, :]
val_labels=val_set[1][0:1000]
N_train=train_data.shape[0]
N_val=val_data.shape[0]
# check data makes sense
plt.imshow(train_data[0,:].reshape(28,28))
print(train_labels[0])

### Generic distance class

We start with a generic trainable distance class that serves as an interface for all the different distances we will implement. The fit method does the training.

In [ ]:
class Distance():
    '''
    trains this distance function on a training set
    '''
    def fit(self, X, y):
        pass
    
    '''
    returns the distance between the sets X1 and X2:
    X1 is n x d (n samples of dimension d)
    X2 is m x d (n samples of dimension d)
    output is n x m (distance matrix)
    '''
    def predict(self, X1, X2):
        pass

## Implementing a $k$-Nearest-Neighbor

<div class="alert alert-success"> <b>Q1.</b> Implement a class that encapsulate the squared euclidean distance ($\| x_1 - x_2 \|^2$) over features projected using a PCA, using the Distance parent class. Test it by comparing the distance matrix before and after projection. <i>(Indicative time: about 10 minutes to code)</i>
</div>

In [ ]:
class PCAL2Distance(Distance):
    def __init__(self, d):
        super().__init__()
        self.d = d
        
    def fit(self, X, y):
    
    def predict(self, X1, X2):

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

<div class="alert alert-success"> <b>Q2.</b> Implement a $k$-NearestNeighbor class that relies on a Distance object to find the neighbors. It also trains the distance. Test it using your PCA based L2 Distance while compressing the features to only 6 dimensions. <i>(Indicative time: about 10 minutes to code, runs in less than 10 seconds)</i>
</div>

In [ ]:
class KNN():
    def __init__(self, distance, k=1):
        self.distance = distance
        self.k = k
    
    '''
    trains the distance and memorizes the training set
    X: n x d (n samples of dimension d)
    y: n (n labels)
    '''
    def fit(self, X, y):
    
    '''
    predict the set of samples
    '''
    def predict(self, X):

In [ ]:
%%time

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

<div class="alert alert-success"> <b>Q3.</b>  Implement a trainable $\ell_2$ distance function relying. I performs a linear projection $P$ such that the distance between similar samples is minimized, trained using gradient descent

$$ \min_P \sum_{x, x_p, y = y_p} \|Px - Px_p\|^2 $$

To reduce the cost of the update, perform the gradient descent on mini-batches of 50 samples taken randomly within the training set. <i>(Indicative time: about 10 minutes to code, runs for about 10 seconds for 2000 iterations)</i>
</div>

In [ ]:
class L2MinDistance(Distance):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.P = np.random.randn(input_dim, output_dim)/jnp.sqrt(input_dim+output_dim)
    
    
    @staticmethod
    def loss(P, X, y):
    
    @staticmethod
    def update(P, X, y, eta=0.01):
        l, dp = jax.value_and_grad(L2MinDistance.loss, argnums=0)(P, X, y)
        return l, P - eta*dp
        
    def fit(self, X, y):
    
    def predict(self, X1, X2):
        

In [ ]:
%%time

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Contrastive loss

Next, we want to mitigate the tendancy of only minimizing the distance to collapse all samples to the same location

<div class="alert alert-success"> <b>Q4.</b> Code a trainable distance function that minimizes the distance between related samples up to a margin and maximizes the distance between unrelated samples up to a margin

$$\min_p \sum_{x, x_p, y = y_p} \max(0, \|Px - Px_p\|^2 - \alpha) - \lambda \sum_{x, x_n, y \neq y_n} \max(0, \beta - \|Px - Px_n\|^2) $$

Plot the loss value as the gradient descent progresses. To reduce the cost, perform the update on mini-batches of 50 samples taken randomly within the training set. <i>(Indicative time: 15 minutes to code, runs in about 15 seconds for 2000 iterations)</i>
</div>

In [ ]:
class L2ContrastiveDistance(Distance):
    def __init__(self, input_dim, output_dim, alpha=0.1, beta= 0.7, lambd=0.1):
        super().__init__()
        self.P = np.random.randn(input_dim, output_dim)/jnp.sqrt(input_dim+output_dim)
        self.alpha = alpha
        self.beta = beta
        self.lambd = lambd
        
    
    @staticmethod
    def loss(P, X, y, alpha, beta, lambd):
    
    @staticmethod
    def update(P, X, y, alpha, beta, lambd, eta=0.05):
        l, dp = jax.value_and_grad(L2ContrastiveDistance.loss, argnums=0)(P, X, y, alpha, beta, lambd)
        return l, P - eta*dp
        
    def fit(self, X, y):
    
    def predict(self, X1, X2):    
        

In [ ]:
%%time

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Triplet loss

Since $k$NN is in nature using a ranking approach by sorting the samples, we will know consider a loss function that enforces the order of the samples rather than their absolute distance values.

$$ \min_P \sum_{x, x_p, x_n, y = y_p, y \neq y_n} \max(0, \alpha + \|x - x_p\|^2 - \|x - x_n\|^2) $$

<div class="alert alert-success"> <b>Q5.</b> Code the triplet loss based trainable distance function. Similarly to other trainable distances, perform the update on mini-batches of size 50. Since there are far more triplets than pairs, it should take longer to converge. <i>(Indicative time: 15 minutes to code, runs in about 1 minute for 5000 iterations)</i>
</div>

In [ ]:
class L2TripletDistance(Distance):
    def __init__(self, input_dim, output_dim, alpha=0.3):
        super().__init__()
        self.P = np.random.randn(input_dim, output_dim)/jnp.sqrt(input_dim+output_dim)
        self.alpha = alpha
    
    @staticmethod
    def loss(P, X, y, alpha):
    
    @staticmethod
    def update(P, X, y, alpha, eta=0.05):
        l, dp = jax.value_and_grad(L2TripletDistance.loss, argnums=0)(P, X, y, alpha)
        return l, P - eta*dp
        
    def fit(self, X, y):
    
    def predict(self, X1, X2):

In [ ]:
%%time

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

<div class="alert alert-success"> <b>Q6.</b> Perform a cross validation on the margin and the number of neighbors. <i>(Indicative time: 10 minutes to code, 10 to 15 minutes to run depending on your grid search size)</i>
</div>

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## Visualization

In order to visualize what is happening during the metric learning, we can interpret the projections of $P$ as images.

<div class="alert alert-warning"> <b>Q7.</b> Extract the projection associated with each output dimension from a trained distance and plot each projector  as an image. Compare to the images obtained from the PCA projectors. <i>(Indicative time: 20 minutes to code, runs instantaneously given trained distances)</i>
</div>

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>